In [1]:
from copy import deepcopy
import numpy as np
from pytreenet.core import Node
from pytreenet.ttns import TreeTensorNetworkState
from pytreenet.operators import Hamiltonian, TensorProduct
from pytreenet.operators.common_operators import pauli_matrices, bosonic_operators
from pytreenet.ttno import SingleTermDiagram, StateDiagram, TreeTensorNetworkOperator
from pytreenet.special_ttn import ForkTreeTensorNetwork
from pytreenet.random import random_tensor_node, crandn


# Tree Tensor Network Operator for the Toy Model
In the main manuscript we consider a toy Hamiltonian to show the workings of our algorithm to find tree tensor network operators from a Hamiltonian. The toy Hamiltonian is defined on a tree structure as given in Fig.4.1a) in the main manuscript as
$$
H_{\text{toy}}= \sum_{j=1}^4 h_j = Y_2 X_3 X_4 + X_1 Y_2 Y_6 + X_1 Y_2 Z_5 + Z_5 X_7 X_8.
$$
Here we want to show that our implemented algorithm yields the same state diagrams as the ones shown in the main text.

In [2]:
# Building the tree
def construct_reference_tree() -> TreeTensorNetworkState:
    """
    Generates the desired tree tensor network used as a reference to construct
     the Hamiltonian.
    """
    ttns = TreeTensorNetworkState()
    # Physical legs come last
    node1, tensor1 = random_tensor_node((1, 1, 2), identifier="site1")
    node2, tensor2 = random_tensor_node((1, 1, 1, 2), identifier="site2")
    node3, tensor3 = random_tensor_node((1, 2), identifier="site3")
    node4, tensor4 = random_tensor_node((1, 2), identifier="site4")
    node5, tensor5 = random_tensor_node((1, 1, 1, 2), identifier="site5")
    node6, tensor6 = random_tensor_node((1, 2), identifier="site6")
    node7, tensor7 = random_tensor_node((1, 1, 2), identifier="site7")
    node8, tensor8 = random_tensor_node((1, 2), identifier="site8")
    ttns.add_root(node1, tensor1)
    ttns.add_child_to_parent(node2, tensor2, 0, "site1", 0)
    ttns.add_child_to_parent(node3, tensor3, 0, "site2", 1)
    ttns.add_child_to_parent(node4, tensor4, 0, "site2", 2)
    ttns.add_child_to_parent(node5, tensor5, 0, "site1", 1)
    ttns.add_child_to_parent(node6, tensor6, 0, "site5", 1)
    ttns.add_child_to_parent(node7, tensor7, 0, "site5", 2)
    ttns.add_child_to_parent(node8, tensor8, 0, "site7", 1)
    return ttns

In [3]:
def construct_toy_hamiltonian() -> Hamiltonian:
    paulis = pauli_matrices()
    conversion_dict = {"X": paulis[0], "Y": paulis[1], "Z": paulis[2], "I2": np.eye(2)}
    term1 = TensorProduct({"site2": "Y", "site3": "X", "site4": "X"})
    term2 = TensorProduct({"site1": "X", "site2": "Y", "site6": "Y"})
    term3 = TensorProduct({"site1": "X", "site2": "Y", "site5": "Z"})
    term4 = TensorProduct({"site5": "Z", "site7": "X", "site8": "X"})
    return Hamiltonian([term1,term2,term3,term4], conversion_dictionary=conversion_dict)

hamiltonian_toy = construct_toy_hamiltonian()

In [4]:
ttns_root1 = construct_reference_tree()
hamiltonian1 = hamiltonian_toy.pad_with_identities(ttns_root1)

### The Single Term Diagrams

In [5]:
for i, term in enumerate(hamiltonian1.terms):
    single_term_diag = SingleTermDiagram.from_single_term(term, ttns_root1)
    print(f"State Diagram for term {i}")
    print(single_term_diag)
    print("-----------------")

State Diagram for term 0
Hyperedges: {'site1': 'I2', 'site2': 'Y', 'site3': 'X', 'site4': 'X', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}
Vertices: [('site1', 'site2'), ('site1', 'site5'), ('site2', 'site3'), ('site2', 'site4'), ('site5', 'site6'), ('site5', 'site7'), ('site7', 'site8')]
-----------------
State Diagram for term 1
Hyperedges: {'site1': 'X', 'site2': 'Y', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'Y', 'site7': 'I2', 'site8': 'I2'}
Vertices: [('site1', 'site2'), ('site1', 'site5'), ('site2', 'site3'), ('site2', 'site4'), ('site5', 'site6'), ('site5', 'site7'), ('site7', 'site8')]
-----------------
State Diagram for term 2
Hyperedges: {'site1': 'X', 'site2': 'Y', 'site3': 'I2', 'site4': 'I2', 'site5': 'Z', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}
Vertices: [('site1', 'site2'), ('site1', 'site5'), ('site2', 'site3'), ('site2', 'site4'), ('site5', 'site6'), ('site5', 'site7'), ('site7', 'site8')]
-----------------
State Diagram for term 3
Hyp

We can see that the single term diagrams are exactly the one we depicted in Fig. 4.1b).

### Complete State Diagram

In [6]:
state_diagram1 = StateDiagram.from_hamiltonian(hamiltonian1, ttns_root1)
print(state_diagram1)

hyperedges:
label = I2; corr_site = site1; connected to ('site1', 'site2'), ('site1', 'site5'), 
label = X; corr_site = site1; connected to ('site1', 'site2'), ('site1', 'site5'), 
label = I2; corr_site = site1; connected to ('site1', 'site2'), ('site1', 'site5'), 
label = Y; corr_site = site2; connected to ('site1', 'site2'), ('site2', 'site3'), ('site2', 'site4'), 
label = Y; corr_site = site2; connected to ('site1', 'site2'), ('site2', 'site3'), ('site2', 'site4'), 
label = I2; corr_site = site2; connected to ('site1', 'site2'), ('site2', 'site3'), ('site2', 'site4'), 
label = X; corr_site = site3; connected to ('site2', 'site3'), 
label = I2; corr_site = site3; connected to ('site2', 'site3'), 
label = X; corr_site = site4; connected to ('site2', 'site4'), 
label = I2; corr_site = site4; connected to ('site2', 'site4'), 
label = I2; corr_site = site5; connected to ('site1', 'site5'), ('site5', 'site6'), ('site5', 'site7'), 
label = I2; corr_site = site5; connected to ('site1', 'sit

One can see that this state diagram corresponds to the state diagram given in Fig. 4.1c) in the main text.

## Considering different Roots
### Root at 5
Here we consider the same tree structure but with the root of the tree at node 5 instead of node 1.

In [7]:
def construct_tree_root_at_5():
    """
    Generates the desired tree tensor network with root at site 5 used as a
     reference to construct the Hamiltonian.
    """
    ttns = TreeTensorNetworkState()

    # Physical legs come last
    node1, tensor1 = random_tensor_node((1, 1, 2), identifier="site1")
    node2, tensor2 = random_tensor_node((1, 1, 1, 2), identifier="site2")
    node3, tensor3 = random_tensor_node((1, 2), identifier="site3")
    node4, tensor4 = random_tensor_node((1, 2), identifier="site4")
    node5, tensor5 = random_tensor_node((1, 1, 1, 2), identifier="site5")
    node6, tensor6 = random_tensor_node((1, 2), identifier="site6")
    node7, tensor7 = random_tensor_node((1, 1, 2), identifier="site7")
    node8, tensor8 = random_tensor_node((1, 2), identifier="site8")

    ttns.add_root(node5, tensor5)
    ttns.add_child_to_parent(node1, tensor1, 0, "site5", 0)
    ttns.add_child_to_parent(node2, tensor2, 0, "site1", 1)
    ttns.add_child_to_parent(node3, tensor3, 0, "site2", 1)
    ttns.add_child_to_parent(node4, tensor4, 0, "site2", 2)
    ttns.add_child_to_parent(node6, tensor6, 0, "site5", 1)
    ttns.add_child_to_parent(node7, tensor7, 0, "site5", 2)
    ttns.add_child_to_parent(node8, tensor8, 0, "site7", 1)
    return ttns

In [8]:
ttns_root5 = construct_tree_root_at_5()
hamiltonian5 = hamiltonian_toy.pad_with_identities(ttns_root5)
state_diagram5 = StateDiagram.from_hamiltonian(hamiltonian5, ttns_root5)
print(state_diagram5)

hyperedges:
label = I2; corr_site = site5; connected to ('site5', 'site1'), ('site5', 'site6'), ('site5', 'site7'), 
label = I2; corr_site = site5; connected to ('site5', 'site1'), ('site5', 'site6'), ('site5', 'site7'), 
label = Z; corr_site = site5; connected to ('site5', 'site1'), ('site5', 'site6'), ('site5', 'site7'), 
label = Z; corr_site = site5; connected to ('site5', 'site1'), ('site5', 'site6'), ('site5', 'site7'), 
label = I2; corr_site = site1; connected to ('site5', 'site1'), ('site1', 'site2'), 
label = X; corr_site = site1; connected to ('site5', 'site1'), ('site1', 'site2'), 
label = I2; corr_site = site1; connected to ('site5', 'site1'), ('site1', 'site2'), 
label = Y; corr_site = site2; connected to ('site1', 'site2'), ('site2', 'site3'), ('site2', 'site4'), 
label = Y; corr_site = site2; connected to ('site1', 'site2'), ('site2', 'site3'), ('site2', 'site4'), 
label = I2; corr_site = site2; connected to ('site1', 'site2'), ('site2', 'site3'), ('site2', 'site4'), 
lab

### Root at 6
Here we consider the same tree structure but with the root of the tree at node 6 instead of node 1. Note that node 6 was a leaf in the other two trees.

In [9]:
def construct_tree_root_at_6():
    """
    Generates the desired tree tensor network with root at site 6 used as a
     reference to construct the Hamiltonian.
    """
    ttns = TreeTensorNetworkState()

    # Physical legs come last
    node1, tensor1 = random_tensor_node((1, 1, 2), identifier="site1")
    node2, tensor2 = random_tensor_node((1, 1, 1, 2), identifier="site2")
    node3, tensor3 = random_tensor_node((1, 2), identifier="site3")
    node4, tensor4 = random_tensor_node((1, 2), identifier="site4")
    node5, tensor5 = random_tensor_node((1, 1, 1, 2), identifier="site5")
    node6, tensor6 = random_tensor_node((1, 2), identifier="site6")
    node7, tensor7 = random_tensor_node((1, 1, 2), identifier="site7")
    node8, tensor8 = random_tensor_node((1, 2), identifier="site8")

    ttns.add_root(node6, tensor6)
    ttns.add_child_to_parent(node5, tensor5, 0, "site6", 0)
    ttns.add_child_to_parent(node1, tensor1, 0, "site5", 1)
    ttns.add_child_to_parent(node2, tensor2, 0, "site1", 1)
    ttns.add_child_to_parent(node3, tensor3, 0, "site2", 1)
    ttns.add_child_to_parent(node4, tensor4, 0, "site2", 2)
    ttns.add_child_to_parent(node7, tensor7, 0, "site5", 2)
    ttns.add_child_to_parent(node8, tensor8, 0, "site7", 1)
    return ttns

In [10]:
ttns_root6 = construct_tree_root_at_6()
hamiltonian6 = hamiltonian_toy.pad_with_identities(ttns_root6)
state_diagram6 = StateDiagram.from_hamiltonian(hamiltonian6, ttns_root6)
print(state_diagram6)

hyperedges:
label = I2; corr_site = site6; connected to ('site6', 'site5'), 
label = Y; corr_site = site6; connected to ('site6', 'site5'), 
label = I2; corr_site = site6; connected to ('site6', 'site5'), 
label = I2; corr_site = site6; connected to ('site6', 'site5'), 
label = I2; corr_site = site5; connected to ('site6', 'site5'), ('site5', 'site1'), ('site5', 'site7'), 
label = I2; corr_site = site5; connected to ('site6', 'site5'), ('site5', 'site1'), ('site5', 'site7'), 
label = Z; corr_site = site5; connected to ('site6', 'site5'), ('site5', 'site1'), ('site5', 'site7'), 
label = Z; corr_site = site5; connected to ('site6', 'site5'), ('site5', 'site1'), ('site5', 'site7'), 
label = I2; corr_site = site1; connected to ('site5', 'site1'), ('site1', 'site2'), 
label = X; corr_site = site1; connected to ('site5', 'site1'), ('site1', 'site2'), 
label = I2; corr_site = site1; connected to ('site5', 'site1'), ('site1', 'site2'), 
label = Y; corr_site = site2; connected to ('site1', 'sit

# Tree Tensor Network Operator for a Spin-Boson model

## Building the Hamiltonian
We consider a Heisenberg chain of length $N$ where each site couples to $M$ non-interacting Bosons, i.e.
$$
H = H_S + H_E + H_{SE}.
$$

$$
H_S = -J\sum_{s=0}^{N-2} \langle \vec{\sigma} , \vec{\sigma} \rangle,
$$
where $J$ is the interaction strength and $\vec{\sigma} = \begin{pmatrix} X & Y & Z \end{pmatrix}^T$ is the vector of Pauli operators.
$$
H_E = \sum_{s=0}^{N-1} \sum_{b=0}^{M-1}\omega_{s,b} B_{s,b}^\dagger B_{s,b},
$$
where $\omega_{s,b}$ is boson's characteristic frequency, and $B$ and $B^\dagger$ are the bosonic annihilation and creation operators.
$$
H_{SE} = \sum_{s=0}^{N-1} \sum_{b=0}^{M-1}\left( g_{s,b} Z_s B_{s,b} + \text{h.c.} \right),
$$
where $g_{s,b}$ is the coupling strength of spin $s$ with boson $(s,b)$.

In [11]:
def gen_hamiltonian(num_spins, num_bosons_per_spin):
    # We use the convention `"spin" + index` for the spin sites
    # and `"bosons_spinindex_bosonindex` for the bosonic sites
    hamiltonian = Hamiltonian()

    # Spin-Spin Interaction
    paulis = ["X", "Y", "Z"]
    for s in range(num_spins-1):
        site_id = "spin_" + str(s)
        neighbour_id = "spin_" + str(s+1)
        for pauli in paulis:
            term = TensorProduct({site_id: "-J" + pauli,
                                    neighbour_id: pauli})
            hamiltonian.add_term(term)

    # Bosonic Site Action
    for s in range(num_spins):
        for b in range(num_bosons_per_spin):
            site_id = "boson_" + str(s) + "_" + str(b)
            term = TensorProduct({site_id: "wN"})
            hamiltonian.add_term(term)

    # Spin-Boson coupling
    for s in range(num_spins):
        spin_identifier = "spin_" + str(s)
        for b in range(num_bosons_per_spin):
            boson_identifier = "boson_" + str(s) + "_" + str(b)
            term = TensorProduct({spin_identifier: "Z",
                                    boson_identifier: "gb+g*b^dagger"})
            hamiltonian.add_term(term)
    return hamiltonian

In [12]:
num_spins = 5
num_bosons_per_spin = 4
hamiltonian = gen_hamiltonian(num_spins, num_bosons_per_spin)
for term in hamiltonian.terms:
    print(term)

{'spin_0': '-JX', 'spin_1': 'X'}
{'spin_0': '-JY', 'spin_1': 'Y'}
{'spin_0': '-JZ', 'spin_1': 'Z'}
{'spin_1': '-JX', 'spin_2': 'X'}
{'spin_1': '-JY', 'spin_2': 'Y'}
{'spin_1': '-JZ', 'spin_2': 'Z'}
{'spin_2': '-JX', 'spin_3': 'X'}
{'spin_2': '-JY', 'spin_3': 'Y'}
{'spin_2': '-JZ', 'spin_3': 'Z'}
{'spin_3': '-JX', 'spin_4': 'X'}
{'spin_3': '-JY', 'spin_4': 'Y'}
{'spin_3': '-JZ', 'spin_4': 'Z'}
{'boson_0_0': 'wN'}
{'boson_0_1': 'wN'}
{'boson_0_2': 'wN'}
{'boson_0_3': 'wN'}
{'boson_1_0': 'wN'}
{'boson_1_1': 'wN'}
{'boson_1_2': 'wN'}
{'boson_1_3': 'wN'}
{'boson_2_0': 'wN'}
{'boson_2_1': 'wN'}
{'boson_2_2': 'wN'}
{'boson_2_3': 'wN'}
{'boson_3_0': 'wN'}
{'boson_3_1': 'wN'}
{'boson_3_2': 'wN'}
{'boson_3_3': 'wN'}
{'boson_4_0': 'wN'}
{'boson_4_1': 'wN'}
{'boson_4_2': 'wN'}
{'boson_4_3': 'wN'}
{'spin_0': 'Z', 'boson_0_0': 'gb+g*b^dagger'}
{'spin_0': 'Z', 'boson_0_1': 'gb+g*b^dagger'}
{'spin_0': 'Z', 'boson_0_2': 'gb+g*b^dagger'}
{'spin_0': 'Z', 'boson_0_3': 'gb+g*b^dagger'}
{'spin_1': 'Z', 'bos

In [13]:
def gen_conversion_dict(boson_phys_dim, mJ, g, w):
    paulis = ["X", "Y", "Z"]
    pauli_ops = pauli_matrices()
    b, bdagger, number_op = bosonic_operators(dimension=boson_phys_dim)
    conversion_dict = {}
    for i, operator in enumerate(pauli_ops):
        conversion_dict[paulis[i]] = operator
        conversion_dict["-J" + paulis[i]] = mJ * operator
    conversion_dict["wN"] = w * number_op
    conversion_dict["gb+g*b^dagger"] = g* (b + bdagger)
    conversion_dict["I2"] = np.eye(2)
    conversion_dict["I" + str(boson_phys_dim)] = np.eye(3)
    return conversion_dict

In [14]:
boson_phys_dim = 3
mJ = -0.4
g = 0.2
w = 0.3
conversion_dict = gen_conversion_dict(boson_phys_dim, mJ, g, w)
conversion_dict

{'X': array([[0.+0.j, 1.+0.j],
        [1.+0.j, 0.+0.j]]),
 '-JX': array([[-0. +0.j, -0.4+0.j],
        [-0.4+0.j, -0. +0.j]]),
 'Y': array([[ 0.+0.j, -0.-1.j],
        [ 0.+1.j,  0.+0.j]]),
 '-JY': array([[-0.+0.j ,  0.+0.4j],
        [-0.-0.4j, -0.+0.j ]]),
 'Z': array([[ 1.+0.j,  0.+0.j],
        [ 0.+0.j, -1.+0.j]]),
 '-JZ': array([[-0.4+0.j, -0. +0.j],
        [-0. +0.j,  0.4-0.j]]),
 'wN': array([[0. , 0. , 0. ],
        [0. , 0.3, 0. ],
        [0. , 0. , 0.6]]),
 'gb+g*b^dagger': array([[0.        , 0.2       , 0.        ],
        [0.2       , 0.        , 0.28284271],
        [0.        , 0.28284271, 0.        ]]),
 'I2': array([[1., 0.],
        [0., 1.]]),
 'I3': array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])}

In [15]:
hamiltonian.conversion_dictionary = conversion_dict

## Matrix Product Form

In [16]:
def gen_mps(num_spins, num_bosons_per_spin, boson_phys_dim):
    spin_tensor = crandn((3,3,2))
    boson_tensor = crandn((3,3,boson_phys_dim))

    mps = TreeTensorNetworkState()
    mps.add_root(Node(identifier="spin_1"), deepcopy(spin_tensor))
    mps.add_child_to_parent(Node(identifier="boson_0_" + str(num_bosons_per_spin-1)),
                            deepcopy(boson_tensor), 0, "spin_1", 0)
    mps.add_child_to_parent(Node(identifier="boson_1_0"), deepcopy(boson_tensor),
                            0, "spin_1", 1)
    for b in reversed(range(0,num_bosons_per_spin-1)):
        left_boson_id = "boson_0_" + str(b)
        left_parent_id = "boson_0_" + str(b+1)
        mps.add_child_to_parent(Node(identifier=left_boson_id),
                                deepcopy(boson_tensor), 0, left_parent_id, 1)
    for b in range(1,num_bosons_per_spin):
        right_boson_id = "boson_1_" + str(b)
        right_parent_id = "boson_1_" + str(b-1)
        mps.add_child_to_parent(Node(identifier=right_boson_id),
                                deepcopy(boson_tensor), 0, right_parent_id, 1)
    mps.add_child_to_parent(Node(identifier="spin_0"), deepcopy(spin_tensor[0,:]),
                            0, "boson_0_0", 1)

    for s in range(2,num_spins):
        spin_identifier = "spin_" + str(s)
        parent_identifier = "boson_" + str(s-1) + "_" + str(num_bosons_per_spin-1)
        mps.add_child_to_parent(Node(identifier=spin_identifier),
                                deepcopy(spin_tensor), 0, parent_identifier, 1)
        for b in range(num_bosons_per_spin):
            boson_identifier = "boson_" + str(s) + "_" + str(b)
            if b == 0:
                parent_identifier = spin_identifier
            else:
                parent_identifier = "boson_" + str(s) + "_" + str(b-1)
            if s == num_spins-1 and b == num_bosons_per_spin-1:
                tensor = deepcopy(boson_tensor[:,0,:])
            else:
                tensor = deepcopy(boson_tensor)
            mps.add_child_to_parent(Node(identifier=boson_identifier),
                                    tensor, 0, parent_identifier, 1)
    return mps

In [17]:
mps = gen_mps(num_spins, num_bosons_per_spin, boson_phys_dim)
mpo_hamiltonian = hamiltonian.pad_with_identities(mps)

## Fork Tree Product Form

In [18]:
def gen_ftps(num_spins, num_bosons_per_spin, boson_phys_dim):
    spin_tensor = crandn((3,3,3,2))
    boson_tensor = crandn((3,3,boson_phys_dim))

    ftps = ForkTreeTensorNetwork(main_identifier_prefix="spin_",
                                    subchain_identifier_prefix="boson_")
    for s in range(num_spins):
        if s == 0:
            tensor = deepcopy(spin_tensor[0,:])
        elif s == num_spins-1:
            tensor = deepcopy(spin_tensor[:,0,:])
        else:
            tensor = deepcopy(spin_tensor)
        ftps.add_main_chain_node(tensor)
    for s in range(num_spins):
        for b in range(num_bosons_per_spin):
            if b == num_bosons_per_spin-1:
                tensor = deepcopy(boson_tensor[:,0,:])
            else:
                tensor = deepcopy(boson_tensor)
            ftps.add_sub_chain_node(tensor, s)
    return ftps

In [19]:
ftps = gen_ftps(num_spins, num_bosons_per_spin, boson_phys_dim)
ftpo_hamiltonian = hamiltonian.pad_with_identities(ftps)

# Star Shape Tree

In [20]:
def gen_sst(num_spins, num_bosons_per_spin, boson_phys_dim):
    spin_shape = [3 for _ in range(num_bosons_per_spin + 2)]
    spin_shape.append(2)
    spin_tensor = crandn(tuple(spin_shape))
    boson_tensor = crandn((3,boson_phys_dim))

    sst = TreeTensorNetworkState()
    sst.add_root(Node(identifier="spin_0"), deepcopy(spin_tensor[0,:]))
    for s in range(1,num_spins):
        if s == num_spins - 1:
            tensor = deepcopy(spin_tensor[0,:])
        else:
            tensor = deepcopy(spin_tensor)
        if s == 1:
            parent_leg = 0
        else:
            parent_leg = 1
        identifier = "spin_" + str(s)
        parent_identifier = "spin_" + str(s-1)
        sst.add_child_to_parent(Node(identifier=identifier), tensor, 0,
                                parent_identifier, parent_leg)
    for s in range(num_spins):
        for b in range(num_bosons_per_spin):
            spin_id = "spin_" + str(s)
            boson_id = "boson_" + str(s) + "_" + str(b)
            tensor = deepcopy(boson_tensor)
            if s == 0 or s == num_spins - 1:
                parent_leg = 1 + b
            else:
                parent_leg = 2 + b
            sst.add_child_to_parent(Node(identifier=boson_id), tensor, 0,
                                    spin_id, parent_leg)
    return sst

In [21]:
sst = gen_sst(num_spins, num_bosons_per_spin, boson_phys_dim)
sst_hamiltonian = hamiltonian.pad_with_identities(sst)

## The Tree Tensor Network Operator

In [22]:
mpo = TreeTensorNetworkOperator.from_hamiltonian(mpo_hamiltonian, mps)
ftpo = TreeTensorNetworkOperator.from_hamiltonian(ftpo_hamiltonian, ftps)
ssto = TreeTensorNetworkOperator.from_hamiltonian(sst_hamiltonian, sst)

In [23]:
for identifier, node in ftpo.nodes.items():
    mpo_shape = mpo.nodes[identifier].shape[:-2]
    ftpo_shape = node.shape[:-2]
    ssto_shape = ssto.nodes[identifier].shape[:-2]
    print(identifier, mpo_shape, ftpo_shape, ssto_shape)

spin_0 (5,) (5, 3) (5, 3, 3, 3, 3)
spin_1 (5, 6) (5, 5, 3) (5, 5, 3, 3, 3, 3)
spin_2 (5, 6) (5, 5, 3) (5, 5, 3, 3, 3, 3)
spin_3 (5, 6) (5, 5, 3) (5, 5, 3, 3, 3, 3)
spin_4 (5, 3) (5, 3) (5, 3, 3, 3, 3)
boson_4_0 (3, 3) (3, 3) (3,)
boson_4_1 (3, 3) (3, 3) (3,)
boson_4_2 (3, 3) (3, 3) (3,)
boson_4_3 (3,) (3,) (3,)
boson_3_0 (6, 6) (3, 3) (3,)
boson_3_1 (6, 6) (3, 3) (3,)
boson_3_2 (6, 6) (3, 3) (3,)
boson_3_3 (6, 5) (3,) (3,)
boson_2_0 (6, 6) (3, 3) (3,)
boson_2_1 (6, 6) (3, 3) (3,)
boson_2_2 (6, 6) (3, 3) (3,)
boson_2_3 (6, 5) (3,) (3,)
boson_1_0 (6, 6) (3, 3) (3,)
boson_1_1 (6, 6) (3, 3) (3,)
boson_1_2 (6, 6) (3, 3) (3,)
boson_1_3 (6, 5) (3,) (3,)
boson_0_0 (6, 5) (3, 3) (3,)
boson_0_1 (6, 6) (3, 3) (3,)
boson_0_2 (6, 6) (3, 3) (3,)
boson_0_3 (5, 6) (3,) (3,)


## Small Example to Check Equality
The above size of the TTN will cause the generation of a very large tensor, if the operators are contracted.
To check that the TTNO actually correspond to the Hamiltonians, we will use an example with fewer sites.

In [24]:
num_spins_small = 3
num_bosons_per_spin_small = 2
boson_phys_dim_small = 3
mJ = -0.4
g = 0.2
w = 0.3

hamiltonian_small = gen_hamiltonian(num_spins_small, num_bosons_per_spin_small)
conversion_dict_small = gen_conversion_dict(boson_phys_dim_small, mJ, g, w)
hamiltonian_small.conversion_dictionary = conversion_dict_small

mps_small = gen_mps(num_spins_small, num_bosons_per_spin_small, boson_phys_dim_small)
mpo_hamiltonian_small = hamiltonian_small.pad_with_identities(mps_small)

ftps_small = gen_ftps(num_spins_small, num_bosons_per_spin_small, boson_phys_dim_small)
ftpo_hamiltonian_small = hamiltonian_small.pad_with_identities(ftps_small)

sst_small = gen_sst(num_spins_small, num_bosons_per_spin_small, boson_phys_dim_small)
sst_hamiltonian_small = hamiltonian_small.pad_with_identities(sst_small)

mpo_small = TreeTensorNetworkOperator.from_hamiltonian(mpo_hamiltonian_small, mps_small)
ftpo_small = TreeTensorNetworkOperator.from_hamiltonian(ftpo_hamiltonian_small, ftps_small)
ssto_small = TreeTensorNetworkOperator.from_hamiltonian(sst_hamiltonian_small, sst_small)

In [25]:
ham_tensor = mpo_hamiltonian_small.to_tensor(mps_small).operator
from_ttno = mpo_small.completely_contract_tree(to_copy=True)[0]
permutation_for_transpose = (0,9,1,10,3,12,5,14,2,11,4,13,6,15,7,16,8,17)
np.allclose(from_ttno, ham_tensor.transpose(permutation_for_transpose))

True

In [26]:
ham_tensor = ftpo_hamiltonian_small.to_tensor(ftps_small).operator
from_ttno = ftpo_small.completely_contract_tree(to_copy=True)[0]
permutation_for_transpose = (0,9,1,10,2,11,7,16,8,17,5,14,6,15,3,12,4,13)
np.allclose(from_ttno, ham_tensor.transpose(permutation_for_transpose))

True

In [27]:
ham_tensor = sst_hamiltonian_small.to_tensor(sst_small).operator
from_ttno = ssto_small.completely_contract_tree(to_copy=True)[0]
permutation_for_transpose = (0,9,1,10,2,11,7,16,8,17,5,14,6,15,3,12,4,13)
np.allclose(from_ttno, ham_tensor.transpose(permutation_for_transpose))

True